## Genelet NAND gate design.
<img align="left" src="Images/NANDgate.png" alt="Genelet Logic Gate" width="700"/>

### Design
The output genelet switch 3 ($\text{Sw}_3$) has two dna activators $\text{A}_31$ and $\text{A}_32$. They both have the same base pairs to complement the promoter region of $\text{Sw}_3$. Addtionally they also have a different sequence of bases for the toehold region (the pink section in $\text{A}_31$ and the light blue section in $\text{A}_32$). Thus in order to sequester them by forming the A-I dna-rna complex, they each need a seperate rna inhibitor strand that that complements their own toehold region ($\text{I}_31$ inhibits $\text{A}_31$ and $\text{I}_32$ inhibits $\text{A}_32$). 

The genelet switches $\text{Sw}_1$ and $\text{Sw}_2$ are regular genelets and their transcripts are the rna strands $\text{I}_31$ and $\text{I}_32$ respectively. 

When neither $\text{Sw}_1$ nor $\text{Sw}_2$ are active, the activator strands for $\text{Sw}_3$ are not sequestered and $\text{Sw}_3$ is fully turned ON equally by $\text{A}_31$ and $\text{A}_32$.

When just $\text{Sw}_1$ is ON and $\text{Sw}_2$ is OFF, $\text{I}_31$ will be produced and $\text{I}_32$ will not be produced. As a result, $\text{I}_31$ will sequester $\text{A}_31$ but $\text{A}_32$ will still be present in abundance and still fully turn ON $\text{Sw}_3$. 

Vice versa as above if $\text{Sw}_2$ is ON and $\text{Sw}_1$ is OFF.

When both $\text{Sw}_1$ and $\text{Sw}_2$ are turned on, $\text{I}_31$ and $\text{I}_32$ are both produced and they sequester both activators $\text{A}_31$ and $\text{A}_32$ thus turning the output $\text{Sw}_3$ fully off.

The logic table as descibed above is that of a NAND gate and should therefore work in theory. The NAND gate can be combined with a NOT gate for AND gate functionality

### Code

In [28]:
from biocrnpyler import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting

In [29]:
class TranscriptionSwitch(Mechanism):
    """
    Reactions involved in this mechanism:
    
    Sw_OFF + A -> Sw_ON
    Sw_ON + I -> Sw_OFF + AI
    A + I -> AI
    Sw_OFF + RNAP <-> Sw_OFF.RNAP -> Sw_OFF + RNAP + P
    Sw_ON + RNAP <-> Sw_ON.RNAP -> Sw_ON + RNAP + P
    AI + RNAseH <-> AI.RNAseH -> A + RNAseH
    
    Optional reactions depending on inputs:
    
    Sw_OFF + A2 -> Sw_ON_2
    Sw_ON_2 + I2 -> Sw_OFF + AI_2
    A2 + I2 -> AI_2
    
    Allows upto 2 sets of activators and inhibitors per genelet
    """
    
    # Set the name and mechanism_type
    def __init__(self, name="transcription_switch", mechanism_type="transcription"):
        
            Mechanism.__init__(self, name=name, mechanism_type=mechanism_type)
    
    def update_species(self, switch_off, transcript, activator, inhibitor, rnap, rnaseH, activator2 = None, inhibitor2 = None, **keywords):
        
        # Return appropriate species depending on whether or not 2nd set of activators and inhibitors are present 
        
        if activator2 != None and inhibitor2 != None:
            return [switch_off, transcript, activator, inhibitor, rnap, rnaseH, activator2, inhibitor2] 
        else:
            return [switch_off, transcript, activator, inhibitor, rnap, rnaseH] 
            
    
    def update_reactions(self, switch_off, transcript, activator, inhibitor, component, part_id, rnap, rnaseH, activator2 = None, inhibitor2 = None, **keywords):
        
        # Create appropriate complex species depending on whether or not 2nd set of activators and inhibitors are present
        
        if activator2 != None and inhibitor2 != None:
            A_I_complex2 = ComplexSpecies([inhibitor2, activator2],name = str(switch_off).replace("_OFF","")+"_AI_2")
            switch_on = ComplexSpecies([switch_off, activator], name = str(switch_off).replace("_OFF","_ON_1"))
            switch_on2 = ComplexSpecies([switch_off, activator2], name = str(switch_off).replace("_OFF","_ON_2"))
        else:     
            switch_on = ComplexSpecies([switch_off, activator], name = str(switch_off).replace("_OFF","_ON"))
        A_I_complex = ComplexSpecies([inhibitor, activator],name = str(switch_off).replace("_OFF","")+"_AI")
        
        
        # Initialise reaction parameters
        
        ktx = component.get_parameter("ktx", part_id = part_id, mechanism = self)
        kleak = component.get_parameter("kleak", part_id = part_id, mechanism = self)
        kdeg = component.get_parameter("kdeg", part_id = part_id, mechanism = self)
        
        ku_tx = component.get_parameter("ku_tx", part_id = part_id, mechanism = self)
        ku_leak = component.get_parameter("ku_leak", part_id = part_id, mechanism = self)
        ku_deg = component.get_parameter("ku_deg", part_id = part_id, mechanism = self)
        
        kM_tx = component.get_parameter("kM_tx", part_id = part_id, mechanism = self)
        kM_leak = component.get_parameter("kM_leak", part_id = part_id, mechanism = self)
        kM_deg = component.get_parameter("kM_deg", part_id = part_id, mechanism = self)
        
        kon = component.get_parameter("kon", part_id = part_id, mechanism = self)
        koff = component.get_parameter("koff", part_id = part_id, mechanism = self)
        ka = component.get_parameter("ka", part_id = part_id, mechanism = self)
            
        kb_tx = (ku_tx + ktx) / kM_tx
        kb_leak = (ku_leak + kleak) / kM_leak
        kb_deg = (ku_deg + kdeg) / kM_deg
        
        #ku_tx = 0.1
        #ku_leak = 0.1
        #ku_deg = 0.1
        
        # Create reactions
        
        reaction_activation_1 = Reaction(inputs = [switch_off, activator], outputs = [switch_on], 
                            k = kon )
        
        reaction_deactivation_1 = Reaction(inputs = [switch_on, inhibitor], outputs = [switch_off, A_I_complex], 
                            k = koff )
        
        reaction_complex_1 = Reaction(inputs = [activator, inhibitor], outputs = [A_I_complex], k = ka)
        
        # Step 1 of transcription and A-I degradation reactions for first set of A/I
        
        reaction_tx_1_1 = Reaction(inputs = [switch_on, rnap], outputs = [ComplexSpecies([rnap, switch_on])], k = kb_tx, k_rev = ku_tx )
        
        reaction_leak_1 = Reaction(inputs = [switch_off, rnap], outputs = [ComplexSpecies([rnap, switch_off])], k = kb_leak, k_rev = ku_leak)
        
        reaction_deg_1_1 = Reaction(inputs = [A_I_complex, rnaseH], outputs = [ComplexSpecies([rnaseH, A_I_complex])], k = kb_deg, k_rev = ku_deg)
        
        # Step 2 of transcription and A-I degradation reactions for first set of A/I
        
        reaction_tx_2_1 = Reaction(inputs = [ComplexSpecies([rnap, switch_on])], outputs = [switch_on, transcript, rnap], k = ktx)
        
        reaction_leak_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_off])], outputs = [switch_off, transcript, rnap], k = kleak)
        
        reaction_deg_2_1 = Reaction(inputs = [ComplexSpecies([rnaseH, A_I_complex])], outputs = [rnaseH, activator], k = kdeg)
        
        # Reactions if second set of activators and inhibitors are present
        
        if activator2 != None and inhibitor2 != None:
            
            reaction_activation_2 = Reaction(inputs = [switch_off, activator2], outputs = [switch_on2], 
                            k = kon )
        
            reaction_deactivation_2 = Reaction(inputs = [switch_on2, inhibitor2], outputs = [switch_off, A_I_complex2], 
                            k = koff )
        
            reaction_complex_2 = Reaction(inputs = [activator2, inhibitor2], outputs = [A_I_complex2], k = ka)
            
            # Step 1 of transcription and A-I degradation reactions for second set of A/I
    
            reaction_tx_1_2 = Reaction(inputs = [switch_on2, rnap], outputs = [ComplexSpecies([rnap, switch_on2])], k = kb_tx, k_rev = ku_tx )
            
            reaction_deg_1_2 = Reaction(inputs = [A_I_complex2, rnaseH], outputs = [ComplexSpecies([rnaseH, A_I_complex2])], k = kb_deg, k_rev = ku_deg)
            
            # Step 2 of transcription and A-I degradation reactions for second set of A/I
        
            reaction_tx_2_2 = Reaction(inputs = [ComplexSpecies([rnap, switch_on2])], outputs = [switch_on2, transcript, rnap], k = ktx)
        
            reaction_deg_2_2 = Reaction(inputs = [ComplexSpecies([rnaseH, A_I_complex2])], outputs = [rnaseH, activator2], k = kdeg)
            
            return [reaction_activation_1, reaction_deactivation_1, reaction_complex_1, reaction_tx_1_1, reaction_tx_2_1, reaction_leak_1, reaction_leak_2,
                reaction_deg_1_1, reaction_deg_2_1, reaction_activation_2, reaction_deactivation_2, reaction_complex_2, reaction_tx_1_2, reaction_tx_2_2,
                reaction_deg_1_2, reaction_deg_2_2]
            
        return [reaction_activation_1, reaction_deactivation_1, reaction_complex_1, reaction_tx_1_1, reaction_tx_2_1, reaction_leak_1, reaction_leak_2,
                reaction_deg_1_1, reaction_deg_2_1]
  


class Genelet(Promoter):
    """
    Genelet switch component using TranscriptionSwitch() mechanism
    Arguments: name, transcript, activator, inhibitor
    Optional Arguments: activator2, inhibitor2, rnap, rnaseH
    """
    def __init__(self, name, transcript, activator, inhibitor, rnap="RNAP", rnaseH="RNAseH", activator2 = None, inhibitor2 = None,  **keywords):
        
        # Set the Regulator
        # Component.set_species(species, material_type = None, attributes = None)
        # is a helper function that allows the input to be a Species, string, or Component.
        
        self.activator = self.set_species(activator, material_type = "dna") 
        self.inhibitor = self.set_species(inhibitor, material_type = "rna")
        self.transcript = self.set_species(transcript, material_type = "rna")
        self.switch_off = self.set_species(str(name)+"_OFF")
        self.rnap = self.set_species(rnap, material_type = "protein")
        self.rnaseH = self.set_species(rnaseH, material_type = "protein")
        
        # Set second activator and inhibitors depending on whether they are present in the input
        
        if activator2 != None and inhibitor2 != None:
            
            self.activator2 = self.set_species(activator2, material_type = "dna") 
            self.inhibitor2 = self.set_species(inhibitor2, material_type = "rna")
        else:
            self.activator2 = None
            self.inhibitor2 = None
        
        custom_mechanisms = {"transcription": TranscriptionSwitch()}
        
        Promoter.__init__(self, name = name, transcript = transcript, mechanisms = custom_mechanisms, **keywords)

    def update_species(self, **keywords):
        
        mech_tx = self.mechanisms["transcription"]
        
        species = [] 
        
        # Call update_species with correct arguments depending on whether the second set of activator and inhibitor are present
        
        if self.activator2 != None and self.inhibitor2 != None:
            species += mech_tx.update_species(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                          rnap = self.rnap, rnaseH = self.rnaseH, activator2 = self.activator2, inhibitor2 = self.inhibitor2)
        else:
            species += mech_tx.update_species(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                          rnap = self.rnap, rnaseH = self.rnaseH)
        
        return species

    def update_reactions(self, **keywords):
        mech_tx = self.mechanisms["transcription"]
        
        reactions = []
        
        # Call update_reactions with correct arguments depending on whether the second set of activator and inhibitor are present
        
        if self.activator2 != None and self.inhibitor2 != None:
            reactions += mech_tx.update_reactions(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                                  rnap = self.rnap, rnaseH = self.rnaseH, activator2 = self.activator2, inhibitor2 = self.inhibitor2,
                                                  component = self, part_id = "Genelet", **keywords)
        else:
            reactions += mech_tx.update_reactions(switch_off = self.switch_off, transcript = self.transcript, activator = self.activator, inhibitor = self.inhibitor, 
                                                  rnap = self.rnap, rnaseH = self.rnaseH, component = self, part_id = "Genelet", **keywords)
        return reactions
    

In [30]:
# Creating CRN for NAND gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

#S4_off = Species("Sw4")

S1 = Genelet(S1_off, transcript = "I31", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I32", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A31", inhibitor = "I31", activator2 = "A32", inhibitor2 = "I32" )
#S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")


repr(M)
CRN = M.compile_crn()
#print(CRN.pretty_print())

In [31]:
# Bioscrape simulation of above CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "protein_RNAseH":20,
      "protein_RNAP":150}
timepoints = np.linspace(0, 5000, 1000)
R = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = io)


bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
#p.circle(timepoints, R["rna_A3"], legend_label = "Activator 3")
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
p.legend.click_policy="hide"
#p.circle(timepoints, R["complex_Sw1_AI"],legend_label = "AI complex 1", color = "green")
#p.legend.location = "center_right"

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 2", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
s.legend.click_policy="hide"
#s.circle(timepoints, R["complex_Sw2_AI"],legend_label = "AI complex 2", color = "green")
#s.legend.location = "center_right"

# Total amount of activated and deactivated forms of Switch 3
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")
#q.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "purple")
q.legend.click_policy="hide"
#q.legend.location = "center_right"

#t = bokeh.plotting.figure(plot_width=300, plot_height=300)
#t.circle(timepoints, R["Sw4_OFF"], legend_label = "OFF switch 4", color = "orange")
#t.circle(timepoints, R["complex_Sw4_ON"], legend_label = "ON switch 4" , color = "red")


r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32" , color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "purple")
r.legend.click_policy="hide"
#r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "AI complex 3" , color = "green")
#p.circle(timepoints, R["complex_rna_A3_rna_I3"],legend_label = "AI complex 3", color = "green")
bokeh.io.show(row(p, s, q,r))
warnings.filterwarnings("ignore")

/home/nazmus/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.2.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:1089: UserWarning: The following species are uninitialized and their value has been defaulted to 0: complex_Sw1_ON, complex_Sw1_AI, complex_Sw1_ON_protein_RNAP, complex_Sw1_OFF_protein_RNAP, complex_Sw1_AI_protein_RNAseH, complex_Sw2_ON, complex_Sw2_AI, complex_Sw2_ON_protein_RNAP, complex_Sw2_OFF_protein_RNAP, complex_Sw2_AI_protein_RNAseH, complex_Sw3_ON_1, complex_Sw3_AI, complex_Sw3_ON_1_protein_RNAP, complex_Sw3_OFF_protein_RNAP, complex_Sw3_AI_protein_RNAseH, complex_Sw3_ON_2, complex_Sw3_AI_2, complex_Sw3_ON_2_protein_RNAP, complex_Sw3_AI_2_protein_RNAseH, 
/home/nazmus/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

Loading BokehJS ...

In [32]:
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"], legend_label = "ON switch 3 - 1" , color = "red")
q.circle(timepoints, R["complex_Sw3_ON_2"], legend_label = "ON switch 3 - 2" , color = "blue")
q.legend.click_policy="hide"
bokeh.io.show(q)